In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import imageio
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

from pathlib import Path
# from fastai import *
# from fastai.vision import *

import tensorflow as tf

print(tf.__version__)

2.2.0


In [2]:
labels = pd.read_csv("./Downloads/train_master.tsv", sep="\t") #教師ラベルの読み込み

In [3]:
labels.head()

,file_name,label_id
0,train_00000.png,11
1,train_00001.png,15
2,train_00002.png,4
3,train_00003.png,14
4,train_00004.png,1


In [4]:
master = pd.read_csv("./Downloads/label_master.tsv", sep="\t") #ラベルマスタの読み込み

In [5]:
master.head()

,label_id,label_name
0,0,aquatic_mammals
1,1,fish
2,2,flowers
3,3,food_containers
4,4,fruit_and_vegetables


In [6]:
sample = pd.read_csv("./Downloads/sample_submit.csv", header=None, sep=",")

In [55]:
sample.head()


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,test_00000.png,0.036151,0.014607,0.034591,0.079934,0.053778,0.011690,0.039329,0.079885,0.047027,...,0.062593,0.037947,0.080989,0.066377,0.020849,0.057315,0.056351,0.033405,0.078505,0.053430
1,test_00001.png,0.033393,0.037818,0.021027,0.043661,0.082885,0.041105,0.069309,0.059735,0.023879,...,0.077569,0.053666,0.044468,0.040490,0.090077,0.050215,0.052718,0.067940,0.030642,0.078394
2,test_00002.png,0.098902,0.047091,0.022525,0.059422,0.038273,0.011877,0.086344,0.054882,0.046612,...,0.028493,0.027183,0.092749,0.076549,0.026914,0.099406,0.032409,0.038710,0.000611,0.060666
3,test_00003.png,0.054705,0.080344,0.047048,0.083701,0.043293,0.068830,0.026523,0.076391,0.008649,...,0.020802,0.054486,0.065079,0.060174,0.045692,0.052243,0.006735,0.036344,0.070731,0.026432
4,test_00004.png,0.087472,0.045188,0.091187,0.003265,0.110322,0.102048,0.048273,0.080005,0.050142,...,0.096636,0.038774,0.050342,0.044085,0.057429,0.023512,0.018324,0.002596,0.030469,0.001106


In [8]:
train_images = []
for fname in labels["file_name"]:
    path = "./Downloads/train_gig/" + fname
    img = imageio.imread(path)
    train_images.append(img)
train_images = np.array(train_images)
print(type(train_images), train_images.shape)

<class 'numpy.ndarray'> (50000, 32, 32, 3)


In [9]:
test_images = []
for fname in sample[0]:
    path = "./Downloads/test_gig/" + fname
    img = imageio.imread(path)
    test_images.append(img)
test_images = np.array(test_images )
print(type(test_images ), test_images.shape)

<class 'numpy.ndarray'> (10000, 32, 32, 3)


In [10]:
train_images = train_images / 255
test_images = test_images / 255

In [11]:
y = labels["label_id"]

y_categorical = utils.to_categorical(y)
y_categorical[0:10,]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [12]:
X_train_image, X_val_image = np.split(train_images, [40000])
y_train_label, y_val_label = np.split(y_categorical, [40000])

In [13]:
inputs = tf.keras.Input(shape=(None, None, 3))
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (160, 160)))(inputs)
x = tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(x)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights='imagenet', input_tensor=x, input_shape=(160, 160, 3),
    include_top=False, pooling='avg'
)

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(20, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 20)                25620     
Total params: 2,283,604
Trainable params: 2,249,492
Non-trainable params: 34,112
_________________________________________________________________


In [19]:
layer_names = [l.name for l in base_model.layers]
idx = layer_names.index('block_12_expand')
print(idx)

110


In [20]:
base_model.trainable = True

for layer in base_model.layers[:idx]:
    layer.trainable = False

In [21]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 20)                25620     
Total params: 2,283,604
Trainable params: 1,825,236
Non-trainable params: 458,368
_________________________________________________________________


In [22]:
model.fit(X_train_image, y_train_label, epochs=1, validation_split=0.2, batch_size=256)

125/125 [==============================] - 1054s 8s/step - loss: 2.5377 - accuracy: 0.2293 - val_loss: 5.2510 - val_accuracy: 0.0506


In [23]:
print(model.evaluate(X_val_image, y_val_label, verbose=0))

[5.215665817260742, 0.04560000076889992]


In [24]:
print(test_images.shape)

predictions = model.predict(test_images)

print(type(predictions))

print(predictions.shape)


(10000, 32, 32, 3)
<class 'numpy.ndarray'>
(10000, 20)


In [26]:
print(predictions[0])

[1.7375313e-02 3.0343321e-01 1.0214645e-03 5.3686790e-02 1.3339324e-03
 1.8290549e-03 3.4014985e-03 5.0057448e-04 4.9680501e-04 2.8190727e-03
 5.2079344e-01 1.2612721e-02 1.2764088e-03 6.8059009e-03 3.5298933e-04
 4.2038327e-03 4.2284377e-02 1.8641753e-02 3.3098290e-04 6.7998054e-03]


In [69]:
file_name = sample[0]
df = pd.DataFrame(predictions)

In [73]:
df_concat = pd.concat([file_name, df],axis=1)

In [75]:
df_concat.head()

,0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,test_00000.png,0.017375,0.303433,0.001021,0.053687,0.001334,0.001829,0.003401,0.000501,0.000497,...,0.520793,0.012613,0.001276,0.006806,0.000353,0.004204,0.042284,0.018642,0.000331,0.006800
1,test_00001.png,0.018297,0.294005,0.001040,0.050353,0.001352,0.002090,0.003657,0.000548,0.000526,...,0.526874,0.013410,0.001413,0.007399,0.000403,0.004523,0.044028,0.019362,0.000383,0.007252
2,test_00002.png,0.017118,0.291644,0.001023,0.053425,0.001321,0.001894,0.003582,0.000517,0.000505,...,0.531862,0.013106,0.001269,0.006741,0.000351,0.004254,0.041935,0.019287,0.000346,0.006878
3,test_00003.png,0.018576,0.271628,0.001044,0.051754,0.001478,0.002454,0.003677,0.000558,0.000577,...,0.545555,0.013845,0.001774,0.008103,0.000458,0.005172,0.044567,0.018488,0.000411,0.007039
4,test_00004.png,0.017738,0.304332,0.001012,0.052868,0.001322,0.001923,0.003459,0.000512,0.000514,...,0.518879,0.012927,0.001371,0.007031,0.000362,0.004379,0.042487,0.018722,0.000350,0.006936


In [77]:
df_concat.to_csv('./Downloads/sample_submit.csv',index = False, header=None)